# Twisted and Tornado: An Alternate Approach

There is another approach to running Twisted code in a notebook, which is to hook into the tornado event loop. [This is in fact an approach taken by Moshe and Glyph in this webinar on YouTube](https://www.youtube.com/watch?v=XXHbXSawwls).

In this notebook I'm going to outline and demo the approach, and show some spots where the behavior of my extension is different/better.

## Setup/Install

Under the hood, IPython uses Tornado, which apparently uses asyncio under the hood. While the linked video and a lot of Google searches will indicate that the installer is in Tornado, it seems that [Tornado uses asyncio under the hood these days](https://github.com/tornadoweb/tornado/issues/2636), which works OK:

In [1]:
from twisted.internet.asyncioreactor import install
install()

Once it's installed, we can import the reactor...

In [2]:
from twisted.internet import reactor

...and we can interact with it directly. For example:

In [3]:
class NoiseMaker():
    tick = 1
    
    def _loop(self):
        if self._running:
            print('LOUD NOISES!')
            reactor.callLater(self.tick, self._loop)
    def start(self):
        self._running = True
        self._loop()
    def stop(self):
        self._running = False

noisemaker = NoiseMaker()

noisemaker.start()

LOUD NOISES!
LOUD NOISES!
LOUD NOISES!
LOUD NOISES!
LOUD NOISES!
LOUD NOISES!
LOUD NOISES!
LOUD NOISES!
LOUD NOISES!
LOUD NOISES!
LOUD NOISES!
LOUD NOISES!
LOUD NOISES!
LOUD NOISES!
LOUD NOISES!
LOUD NOISES!
LOUD NOISES!
LOUD NOISES!
LOUD NOISES!
LOUD NOISES!


IPython evaluates the code block and exits, and our `NoiseMaker` continues printing in the background.

Note, however, that IPython can't know where the printed text is coming from, so it dumps the output to whichever cell you last evaluated. For example, if I evaluate the following cell, it will start echoing `LOUD NOISES!`:

In [4]:
print('shhh, this is a library')

shhh, this is a library
LOUD NOISES!
LOUD NOISES!
LOUD NOISES!
LOUD NOISES!
LOUD NOISES!


(we'll stop the noisemaker for now)

In [5]:
noisemaker.stop()

That there isn't a magical library that will fix this problem. If you were to do this inside `run_in_reactor` with Crochet, it would have the same behavior.

This isn't the worst thing. Twisted doesn't print to screen by default and in many cases spinning up a server in the background is exactly what you want. If you write code that doesn't run forever in the background like our `NoiseMaker` and you run the cell manually, the output will show up in the right cell in general. It's even possible to get Jupyter to pretty-print objects by calling `display` directly:



In [6]:
class Scream:
    def __init__(self, value):
        self.value = value
    def _repr_markdown_(self):
        return f'# {self.value}'

reactor.callLater(1, display, Scream('AHHHHHHH'))

<DelayedCall 0x7f4afc2bd6d8 [0.9759202460118104s] called=0 cancelled=0 AsyncioSelectorReactor.callLater.<locals>.run()>

# AHHHHHHH

That said, the trick to getting useful output that's guaranteed to stay in its cell is to get IPython to understand where a series of work begins and ends. For a background "service" like our `NoiseMaker`, we don't have a lot of great options. For a lot of code, however, we could signal the end of a series of actions with the firing of a Deferred.

IPython doesn't support instrumenting Deferreds as such, but it does have some support for running async/await code. If we can get IPython to run Twisted async/await code then we should largely be in business. However, the default event loop used for awaited code, the one from asyncio, doesn't know how to handle awaited Deferreds, so trying this will cause an exception:

In [7]:
from twisted.internet.defer import Deferred

def sleep(t):
    d = Deferred()
    reactor.callLater(t, d.callback, None)
    return d

In [8]:
print('Going to sleep...')

await sleep(1)

print('Waking up!')

Going to sleep...


RuntimeError: Task got bad yield: <Deferred at 0x7f4afcbd6630>

## So what to do?

There are a few options:

1. Use [my project](https://github.com/jfhbrook/twisted_ipython/blob/master/README.ipynb) and get async/await in return for having a tougher time calling non-awaited code.
2. Use the approach lined out in this notebook and forego async/await in return for being able to interact with the reactor directly.

Both of these approaches have advantages and drawbacks and you'll have to weigh them when making this choice.